In [ ]:
import os
import warnings
import pandas as pd
import panel as pn
import numpy as np
import datetime as dt
import holoviews as hv
from colorcet import fire
from holoviews.operation.datashader import rasterize
from bokeh.models.tools import HoverTool
from scripts.sathelpers import SatelliteDataStore
hv.extension('bokeh')

In [ ]:
# Filter warnings in hit intersection code
warnings.filterwarnings('ignore', category=RuntimeWarning)

In [ ]:
# Set some configuration variables
# In general, these should be explicit paths with no variables or homedir (~)
AIS_DIR = "data/Cleaned_AIS"
SAT_DIR = "data/index_active"

if not os.path.isdir(AIS_DIR) or not os.path.isdir(SAT_DIR):
    raise IOError("Invalid source data directory")

## Step 0. Configure the input parameters

In [ ]:
# Based on the year of interest, also define the AIS file to look at
AIS_FILENAME = "ais_2015.h5"

## Step 1. Load the satellite data

In [ ]:
satdata = SatelliteDataStore(SAT_DIR)

In [ ]:
df = pd.read_csv("metadata/UCS-Satellite-Database-8-1-2020.txt", sep='\t', encoding='L1', low_memory=False) 
df = df.dropna(axis='columns',how='all')
df.head(3)

In [ ]:
norad_names = dict(zip(df['Name of Satellite, Alternate Names'], df['NORAD Number']))
available_norad_ids = satdata.get_norad_ids()
norad_names.pop([el for el in list(norad_names.keys()) if type(el) != str][0]) # Drop nan record
norad_names = {k:int(v) for k,v in norad_names.items() if int(v) in available_norad_ids}

## Step 2. Load the AIS data

Since the example in this notebook is from the period of time of 2009, we just need to load its AIS tracks.

In [ ]:
ais = pd.read_hdf(os.path.join(AIS_DIR, AIS_FILENAME))
ais.sort_values(by="date_time", inplace=True)
ais.info()

## Step 3. Compute the visible points

In [ ]:
from scripts import intersect; intersect.PRINT_INFO=False

## Step 4. Visualize the results

Start by defining date pickers:

In [ ]:
# Vessel metadata CSV files for Zones 1,2 and 3 in 2015
vessel_info_dict = {}
try:
    vessel_info_z1 = pd.read_csv("Vessel_Zone1.csv")
    vessel_info_z2 = pd.read_csv("Vessel_Zone2.csv")
    vessel_info_z3 = pd.read_csv("Vessel_Zone3.csv")
    for _, row in list(vessel_info_z1.iterrows()) + list(vessel_info_z2.iterrows()) + list(vessel_info_z3.iterrows()):
        vessel_info_dict[int(row['mmsi_id'])] = {'vessel_name':row['vessel_name'], 'length':row['length'], 
                                             # Problematic due to uninterpretable vessel int ids
                                             # 'vessel_type':int(row['vessel_type'])
                                             'width':row['width']
                                             }
except: pass

### Utility functions

In [ ]:
def modulo_lon(val):
    return (val+180) % 360 - 180

def get_track(lat, lon, lat_clip=85.5):
    "Turn track of latitudes and longitudes into NaN-separated Curve"
    mask = np.abs(lat) > lat_clip
    lat[mask] = np.float('nan')
    lon[mask] = np.float('nan')
    lon = np.array([modulo_lon(el) for el in lon])
    
    eastings, northings = hv.util.transform.lon_lat_to_easting_northing(lon,lat)
    # Heuristic to insert NaNs to break up Curve (prevent wrapping issues at date line)
    inds = np.where(np.abs(np.diff(eastings)) > 2e7)[0] # Big delta to split on
    inds += 1
    eastings  = np.insert(eastings,  inds, [float('nan') for i in range(len(inds))])
    northings = np.insert(northings, inds, [float('nan') for i in range(len(inds))])
    return hv.Curve((eastings, northings))


def get_vessels(hits, start_date, end_date):
    "Mark the vessels in the AIS data at the midpoint between start and end date"
    sdate = dt.datetime(start_date.year, start_date.month, start_date.day)
    edate = dt.datetime(end_date.year, end_date.month, end_date.day)
    middate = sdate + (edate - sdate) / 2
    lats, lons, lengths, widths, vessel_names = [], [], [], [], []
    for mmsi_id, df in hits.groupby('mmsi_id'):
        df['timestamp'] = pd.to_datetime(df['date_time'])
        df = df.drop_duplicates().set_index('timestamp') # Assuming sorted avoiding .sort_values(by='timestamp')
        idx = df.index.get_loc(middate, method='nearest')
        vinfo = vessel_info_dict.get(mmsi_id, {})
        vessel_names.append(vinfo.get('vessel_name', 'Unknown'))
        lengths.append(vinfo.get('length', 'Unknown'))
        widths.append(vinfo.get('width', 'Unknown'))
        lats.append(float(df.iloc[idx]['lat']))
        lons.append(float(df.iloc[idx]['lon']))
        
    eastings, northings = hv.util.transform.lon_lat_to_easting_northing(np.array(lons),np.array(lats))
    tooltips = [("name", "@name"), ("latitude", "@lat"), ("longitude", "@lon"),
                ("length", "@length"), ("width", "@width")]
    return hv.Points((eastings, northings, vessel_names, lengths, widths, lats, lons), 
                     vdims=['name', 'length', 'width', 'lat', 'lon']).opts(color='white', size=4,  marker='triangle', 
                                                            tools=[HoverTool(tooltips=tooltips)])

DynamicMap callback:

In [ ]:
def rasterize_hits(name_dict, start_dict, end_dict, start_hours_dict, end_hours_dict,
                   checkbox_dict, plot_size_dict, rangexy_dict):
    "DynamicMap callback plotting rasterized hits, satellite track and vessel locations"
    name, start_date, end_date = name_dict['value'], start_dict['value'], end_dict['value']
    start_hours, end_hours = start_hours_dict['value'], end_hours_dict['value']
    full_range = checkbox_dict['value']
    norad_id = int(norad_names[name])
    start_time = pd.Timestamp(year=start_date.year, month=start_date.month, day=start_date.day,
                              hour = start_hours.hour, minute=start_hours.minute, second=start_hours.second)
    
    end_time = pd.Timestamp(year=end_date.year, month=end_date.month, day=end_date.day,
                            hour = end_hours.hour, minute=end_hours.minute, second=end_hours.second)
    if full_range:
        start_time, end_time = satdata.get_timespan(norad_id)
    try:
        (times, lats, lons, alts) = satdata.get_precomputed_tracks(norad_id, start=start_time, end=end_time)
    except: 
        print('Exception in get_precomputed_tracks: %s' % str(e))
        return hv.Overlay([])

    # Need longitudes in (-180,180) format, not 0-360
    mask = lons > 180.0
    lons[mask] -= 360  
    
    try:
        sat = pd.DataFrame({"date_time": times.astype("<M8[s]"),"lat": lats, "lon": lons, "alt": alts})
        hits = intersect.compute_hits(sat, ais, start_time=str(start_date), end_time=str(end_date), workers=4)
    except Exception as e:
        print('Exception in compute_hits: %s' % str(e))
        return hv.Overlay()

    mask = (np.abs(hits['lat']) < 85)
    eastings, northings = hv.util.transform.lon_lat_to_easting_northing(hits['lon'], hits['lat'])
    rasterim = rasterize(hv.Points(pd.DataFrame({'northing':northings[mask], 
                    'easting':eastings[mask]}), ['easting', 'northing']),
                             width = int(plot_size_dict['width']), height = int(plot_size_dict['height']),
                             x_range=rangexy_dict['x_range'], y_range=rangexy_dict['y_range'], dynamic=False
                            ).opts(cmap=fire[180:], width=700, height=500, cnorm='eq_hist', alpha=0.5)

    elements = [rasterim]
    if not full_range:
        elements += [get_track(lats, lons).opts(color='red'),
                     get_vessels(hits, start_date, end_date)]
    return hv.Overlay(elements)

### Declaring panel widgets

Satellite selector (autocomplete input widget):

In [ ]:
selector = pn.widgets.AutocompleteInput(
    name='Satellite', options=list(norad_names.keys()),
    placeholder='Satellite name')
selector.value = 'International Space Station (ISS [first element Zarya])'

Date and checkbox widgets:

In [ ]:
start_date = pn.widgets.DatePicker(name='Start Date', value=dt.date(2015, 1, 1))
end_date = pn.widgets.DatePicker(name='End Date', value=dt.date(2015, 1, 4))
full_range = pn.widgets.Checkbox(name='Full date range')

Time widgets:

In [ ]:
zero_hours = dt.datetime(2020, 1, 1, 0, 0, 0, 0)
twelve_hours = dt.datetime(2020, 1, 1, 12, 0, 0, 0)
start_time =pn.widgets.DatetimeInput(value=zero_hours, format="%H:%M")
end_time = pn.widgets.DatetimeInput(value=twelve_hours, format="%H:%M")

Setting up callback to disable date pickers when 'full date range' checkbox active:

In [ ]:
@pn.depends(full_range.param.value, watch=True)
def disable_callback(full_range):
    start_date.disabled = full_range
    end_date.disabled = full_range

### Declaring HoloViews elements

In [ ]:
tiles = hv.element.tiles.ESRI().redim(x='easting', y='northing')
hits_dmap = hv.DynamicMap(rasterize_hits, 
                          streams=[selector.param.value,  start_date.param.value, end_date.param.value,
                                   start_time.param.value, end_time.param.value,
                                   full_range.param.value,
                                    hv.streams.PlotSize(width=700, height=500),  hv.streams.RangeXY()],
                     positional_stream_args=True)

### Declaring Panel dashboard

In [ ]:
pn.Column(pn.Column(full_range, 
                    pn.Row(start_date, end_date),
                    pn.Row(start_time, end_time)), selector, tiles * hits_dmap).servable()